In [ ]:
import pandas as pd
import numpy as np
import glob  # 파일 경로를 쉽게 찾기 위한 라이브러리
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 1. 파일 경로 설정
# 예: '/content/drive/My Drive/github_project/'
path = '/content/drive/My Drive/github_project/'

# path 경로에 있는 'github_202*.csv' 패턴의 모든 파일 목록 가져오기
all_files = glob.glob(path + "github_202*.csv")
all_files.sort()  # 2021, 2022... 순서대로 정렬

# 2. 모든 CSV 파일을 읽어와서 리스트에 저장
df_list = []
for filename in all_files:
    df = pd.read_csv(filename)
    print(f"'{filename.split('/')[-1]}' 로드 완료 (행: {len(df)})")
    df_list.append(df)

# 3. 모든 데이터프레임을 하나로 병합
github_df = pd.concat(df_list, ignore_index=True)

print(f"\n--- 총 {len(df_list)}개 GitHub 파일 병합 완료 ---")
print(f"병합 후 총 행: {len(github_df)}")
print(github_df.head()) # 상위 5개 행 확인

'github_2021.csv' 로드 완료 (행: 160870)
'github_2022.csv' 로드 완료 (행: 177318)
'github_2023.csv' 로드 완료 (행: 375469)
'github_2024.csv' 로드 완료 (행: 460788)
'github_2025.csv' 로드 완료 (행: 685061)

--- 총 5개 GitHub 파일 병합 완료 ---
병합 후 총 행: 1859506
          id                  name                        full_name  \
0  326059244              mirotalk         miroslavpejic85/mirotalk   
1  326020989  TerrainAndAIPractice  tyler-lady/TerrainAndAIPractice   
2  325951888    superpowers-skills    xitachixxx/superpowers-skills   
3  326066116       best-of-jupyter       ml-tooling/best-of-jupyter   
4  326002236                   ViT   FrancescoSaverioZuppichini/ViT   

                        owner    owner_type  \
0             miroslavpejic85          User   
1                  tyler-lady          User   
2                  xitachixxx          User   
3                  ml-tooling  Organization   
4  FrancescoSaverioZuppichini          User   

                                         description    langua

In [ ]:
print("--- GitHub 데이터 컬럼 목록 ---")
print(github_df.columns)

--- GitHub 데이터 컬럼 목록 ---
Index(['id', 'name', 'full_name', 'owner', 'owner_type', 'description',
       'language', 'stars', 'forks', 'watchers', 'open_issues', 'created_at',
       'updated_at', 'pushed_at', 'size', 'url', 'topics', 'license',
       'default_branch', 'has_wiki', 'has_pages', 'archived'],
      dtype='object')


In [ ]:
# 'id' 컬럼을 기준으로 중복 제거
id_column = 'id'

print(f"중복 제거 전 행: {len(github_df)}")
# id_column 기준으로 중복된 행 제거
github_df.drop_duplicates(subset=[id_column], keep='first', inplace=True)
print(f"중복 제거 후 행: {len(github_df)}")

중복 제거 전 행: 1859506
중복 제거 후 행: 1859506


In [ ]:
# 결측치 현황 파악 (컬럼별 null 개수)
print("\n--- 결측치 현황 ---")
print(github_df.isnull().sum())

# --- 결측치 처리 예시 (컬럼명 확인 후 실행) ---
# 텍스트 컬럼인 'description'과 'topics'는 빈 문자열로 채움
github_df['description'] = github_df['description'].fillna('')
github_df['topics'] = github_df['topics'].fillna('')

github_df['forks'] = github_df['forks'].fillna(0)
github_df['stars'] = github_df['stars'].fillna(0)

print("\n--- 주요 컬럼 결측치 처리 완료 ---")


--- 결측치 현황 ---
id                      0
name                    3
full_name               0
owner                   0
owner_type              0
description        571203
language           329273
stars                   0
forks                   0
watchers                0
open_issues             0
created_at              0
updated_at              0
pushed_at               6
size                    0
url                     0
topics            1617195
license                 0
default_branch          1
has_wiki                0
has_pages               0
archived                0
dtype: int64

--- 주요 컬럼 결측치 처리 완료 ---


In [ ]:
# 1. 날짜 컬럼을 datetime 객체로 변환
date_column = 'created_at'
github_df[date_column] = pd.to_datetime(github_df[date_column], errors='coerce')

# 2. 기준 날짜 (2022-11-30) - [수정됨]
# utc=True를 추가
cutoff_date = pd.to_datetime('2022-11-30', utc=True)

# 3. 'period' 컬럼 생성
github_df['period'] = np.where(
    (github_df[date_column].notnull()) & (github_df[date_column] > cutoff_date),
    'After',
    'Before'
)

print("\n--- 'period' 변수 생성 결과 ---")
print(github_df['period'].value_counts())


--- 'period' 변수 생성 결과 ---
period
After     1539213
Before     320293
Name: count, dtype: int64


In [ ]:
# 'description'과 'topics' 컬럼을 합쳐서 검색용 텍스트 생성
github_df['search_text'] = github_df['description'].astype(str) + ' ' + \
                           github_df['topics'].astype(str)

# 검색을 위해 모두 소문자로 변환
search_text_lower = github_df['search_text'].str.lower()

# --- 2. 'ai_field' 분류 ---
conditions_ai_field = [
    search_text_lower.str.contains('llm|large language model|gpt|llama|bert', case=False),
    search_text_lower.str.contains('deep learning|neural network|cnn|rnn|tensorflow|pytorch|keras', case=False),
    search_text_lower.str.contains('computer vision|opencv|yolo|image recognition', case=False),
    search_text_lower.str.contains('nlp|natural language processing|transformer|hugging face', case=False),
    search_text_lower.str.contains('machine learning|scikit-learn|ml|xgboost', case=False)
]
choices_ai_field = ['LLM', 'Deep Learning', 'Computer Vision', 'NLP', 'General ML']
github_df['ai_field'] = np.select(conditions_ai_field, choices_ai_field, default='Others')

# --- 3. 'keyword' 분류 ---
conditions_keyword = [
    search_text_lower.str.contains('ai agent|auto-gpt|babyagi', case=False),
    search_text_lower.str.contains('autonomous', case=False),
    search_text_lower.str.contains('vibe coding', case=False)
]
choices_keyword = ['AI Agent', 'Autonomous', 'Vibe Coding']
github_df['keyword'] = np.select(conditions_keyword, choices_keyword, default='None')

print("\n--- 'ai_field' 분류 결과 ---")
print(github_df['ai_field'].value_counts())
print("\n--- 'keyword' 분류 결과 ---")
print(github_df['keyword'].value_counts())


--- 'ai_field' 분류 결과 ---
ai_field
Others             934506
General ML         375296
Deep Learning      265546
LLM                256161
NLP                 18728
Computer Vision      9269
Name: count, dtype: int64

--- 'keyword' 분류 결과 ---
keyword
None           1815187
AI Agent         37274
Autonomous        6838
Vibe Coding        207
Name: count, dtype: int64


In [ ]:
# 4. 최종 결과 저장 (path는 2단계에서 설정한 경로 사용)
output_path = path + 'github_all_years_cleaned.csv'
github_df.to_csv(output_path, index=False)

print(f"\n전처리 및 파생 변수 생성이 완료된 파일이")
print(f"'{output_path}'에 저장되었습니다.")


전처리 및 파생 변수 생성이 완료된 파일이
'/content/drive/My Drive/github_project/github_all_years_cleaned.csv'에 저장되었습니다.
